#### DMV Airline Delays
#### Twitter @sebbystats
#### https://github.com/asranasinghe
<br>

This notebook compiles statistics and graphics to summarize flight on-time performance.

Data pulled on January 11th, 2024 from the Bureau of Transportation Statistics (BTS), Marketing Carrier On-Time Performance (Beginning January 2018) database, found __[here](https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGK&QO_fu146_anzr=b0-gvzr)__

In [89]:
import pandas as pd
import numpy as np
import os
import datetime as dt

#### Set Required Inputs

In [323]:
# [USER INPUT] Set path to metadata, and folder containing raw .csv files downloaded from BTS. Set columns as needed

data_dir = '/Users/sebastianranasinghe/Documents/Research/DMV Airline Delays/data/raw'
columns = ['FL_DATE','MKT_UNIQUE_CARRIER','BRANDED_CODE_SHARE','MKT_CARRIER_FL_NUM','ORIGIN','DEST','CRS_DEP_TIME','DEP_TIME','DEP_DELAY_NEW','CRS_ARR_TIME','ARR_TIME','ARR_DELAY_NEW','CANCELLED','CANCELLATION_CODE','DIVERTED','CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME','DISTANCE','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY','DIV_AIRPORT_LANDINGS','DIV_REACHED_DEST','DIV_ACTUAL_ELAPSED_TIME','DIV_ARR_DELAY','DIV_DISTANCE']

al_names = '/Users/sebastianranasinghe/Documents/Research/DMV Airline Delays/data/metadata/L_UNIQUE_CARRIERS.csv'
al_names_df = pd.read_csv(al_names)

# [USER INPUT]: Choose departure airports

airports = ['IAD', 'BWI', 'DCA']
# airports = list(al_names_df.Code) # For All Airports


# [USER INPUT]: Filter flights based On date range

start_date = dt.date(2022, 1, 1)
end_date = dt.date(2023, 12, 31)


# [USER INPUT]: Cancellation Multiplier

cancel_mult = 3


# [USER INPUT]: Delay Threshold i.e. number of minutes allowed between expected and actual arrival time before a flight is considered delayed

delay_thres = 15


#### Load + Prep Data

In [352]:
# Load metadata

al_names_df = pd.read_csv(al_names)
al_names_df.columns = ['CODE', 'CARRIER_DETAIL']


# Load and concatenate flight data

flights = pd.DataFrame(columns = columns)

for file in os.listdir(data_dir)[:2]:

    file_path = os.path.join(data_dir, file)
    
    tmp = pd.read_csv(file_path)
    tmp_rows = tmp.shape[0]

    flights = pd.concat([flights, tmp], axis = 0)
    print('Loaded {} rows from {}'.format(tmp_rows, file))


# Additional transforms, filterin

flights['ARR_DELAY_IND'] = np.where(pd.isna(flights['ARR_DELAY_NEW']) == False, \
                                np.where(flights.ARR_DELAY_NEW >= delay_thres, 1, 0), \
                                np.nan)

flights = flights.loc[flights['DIV_REACHED_DEST'] != 0] # Filter out flights that were diverted AND did not reach final destination
flights['FL_DATE'] = pd.to_datetime(flights.FL_DATE).dt.date
flights = pd.merge(flights, al_names_df, how = 'inner', left_on = 'MKT_UNIQUE_CARRIER', right_on = 'CODE')


# Filter based on user inputs

flights = flights.loc[(flights['ORIGIN'].isin(airports)) \
           & (flights['FL_DATE'] >= start_date) \
           & (flights['FL_DATE'] <= end_date)]

           
print('\n')
print(flights.shape)
print(flights.columns)
flights.head()


Loaded 613577 rows from T_ONTIME_MARKETING_062023.csv
Loaded 602057 rows from T_ONTIME_MARKETING_062022.csv


(51944, 31)
Index(['FL_DATE', 'MKT_UNIQUE_CARRIER', 'BRANDED_CODE_SHARE',
       'MKT_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY_NEW',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY',
       'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'DIV_AIRPORT_LANDINGS', 'DIV_REACHED_DEST', 'DIV_ACTUAL_ELAPSED_TIME',
       'DIV_ARR_DELAY', 'DIV_DISTANCE', 'ARR_DELAY_IND', 'CODE',
       'CARRIER_DETAIL'],
      dtype='object')


,FL_DATE,MKT_UNIQUE_CARRIER,BRANDED_CODE_SHARE,MKT_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,CRS_ARR_TIME,...,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,ARR_DELAY_IND,CODE,CARRIER_DETAIL
2,2023-06-01,AA,AA,1002,BWI,CLT,829,816.0,0.0,1004,...,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,AA,American Airlines Inc.
22,2023-06-01,AA,AA,1029,DCA,ORD,1130,1130.0,0.0,1241,...,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,AA,American Airlines Inc.
116,2023-06-01,AA,AA,1158,DCA,DFW,1859,1850.0,0.0,2118,...,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,AA,American Airlines Inc.
137,2023-06-01,AA,AA,1177,DCA,DFW,1336,1334.0,0.0,1555,...,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,AA,American Airlines Inc.
139,2023-06-01,AA,AA,1178,DCA,MIA,630,623.0,0.0,915,...,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,AA,American Airlines Inc.


#### Analyze

In [365]:
# Find percentage of flights that arrives late (longer than 15 minutes), by airline

al_delay_perc = flights[['CARRIER_DETAIL', 'ARR_DELAY_IND']].value_counts().reset_index()
al_delay_perc.columns = ['CARRIER_DETAIL', 'ARR_DELAY_IND', 'COUNT']

summary = al_delay_perc.pivot(index = 'CARRIER_DETAIL', columns = 'ARR_DELAY_IND', values = 'COUNT').reset_index()
summary.columns = ['CARRIER_DETAIL', 'ON_TIME', 'DELAYED']


# Find number of flights cancelled by carrier

cancellations = flights.groupby(['CARRIER_DETAIL']).agg({'CANCELLED': sum}).reset_index()
cancellations.columns = ['CARRIER_DETAIL', 'CANCELLED']

summary = pd.merge(summary, cancellations, on = 'CARRIER_DETAIL', how = 'outer')


# Find number of flights diverted by carrier (only includes flights that were diverted and made it to expected final destination)

diversions = flights.groupby(['CARRIER_DETAIL']).agg({'DIVERTED': sum}).reset_index()
diversions.columns = ['CARRIER_DETAIL', 'DIVERTED']

summary = pd.merge(summary, diversions, on = 'CARRIER_DETAIL', how = 'outer')


# Arrival Delay (Minutes)

al_delay_total = flights[['CARRIER_DETAIL', 'ARR_DELAY_NEW']].loc[flights['ARR_DELAY_IND'] == 1]

al_delay_total = al_delay_total.groupby(['CARRIER_DETAIL']).agg({'ARR_DELAY_NEW': sum}).reset_index()
al_delay_total.columns = ['CARRIER_DETAIL', 'ARRIVAL_DELAY_TOTAL']

summary = pd.merge(summary, al_delay_total, on = 'CARRIER_DETAIL', how = 'outer')


# Diversion Delay (Minutes)

al_div_delay_total = flights[['CARRIER_DETAIL', 'DIV_ARR_DELAY']]

al_div_delay_total = al_div_delay_total.groupby(['CARRIER_DETAIL']).agg({'DIV_ARR_DELAY': sum}).reset_index()
al_div_delay_total.columns = ['CARRIER_DETAIL', 'DIVERSION_DELAY_TOTAL']

summary = pd.merge(summary, al_div_delay_total, on = 'CARRIER_DETAIL', how = 'outer')


# Get Total Flights, Total Delay

summary['ALL_FLIGHTS'] = summary.ON_TIME + summary.DELAYED + summary.CANCELLED + summary.DIVERTED
summary['TOTAL_DELAY'] = summary.ARRIVAL_DELAY_TOTAL + summary.DIVERSION_DELAY_TOTAL


# Get Total Delay for flights that were delayed / diverted

summary['DEL_DIV'] = summary['DELAYED'] + summary['DIVERTED']
summary['DEL_DIV_AVG'] = summary['TOTAL_DELAY'] / summary['DEL_DIV']


summary.head()

,CARRIER_DETAIL,ON_TIME,DELAYED,CANCELLED,DIVERTED,ARRIVAL_DELAY_TOTAL,DIVERSION_DELAY_TOTAL,ALL_FLIGHTS,TOTAL_DELAY,DEL_DIV,DEL_DIV_AVG
0,Alaska Airlines Inc.,506,197,7.0,8.0,14521.0,1310.0,718.0,15831.0,205.0,77.224390
1,Allegiant Air,73,47,3.0,0.0,5187.0,0.0,123.0,5187.0,47.0,110.361702
2,American Airlines Inc.,11095,3708,769.0,58.0,294014.0,17561.0,15630.0,311575.0,3766.0,82.733670
3,Delta Air Lines Inc.,3907,978,247.0,5.0,95326.0,741.0,5137.0,96067.0,983.0,97.728383
4,Frontier Airlines Inc.,334,137,14.0,1.0,12098.0,114.0,486.0,12212.0,138.0,88.492754


In [366]:
# Calculcate Unreliable Adjusted Score, penalizes flight cancellations by a factor of 'cancel_mult'

summary['UNRELIABLE_ADJ'] = summary['DELAYED'] + cancel_mult * summary['CANCELLED'] + summary['DIVERTED']
summary['ALL_FLIGHTS_ADJ'] = summary['ON_TIME'] + summary['DELAYED'] + cancel_mult * summary['CANCELLED'] + summary['DIVERTED']

summary['ALL_FLIGHTS_ADJ_PERC'] = summary['UNRELIABLE_ADJ'] / summary['ALL_FLIGHTS_ADJ']

summary.head()

,CARRIER_DETAIL,ON_TIME,DELAYED,CANCELLED,DIVERTED,ARRIVAL_DELAY_TOTAL,DIVERSION_DELAY_TOTAL,ALL_FLIGHTS,TOTAL_DELAY,DEL_DIV,DEL_DIV_AVG,UNRELIABLE_ADJ,ALL_FLIGHTS_ADJ,ALL_FLIGHTS_ADJ_PERC
0,Alaska Airlines Inc.,506,197,7.0,8.0,14521.0,1310.0,718.0,15831.0,205.0,77.224390,226.0,732.0,0.308743
1,Allegiant Air,73,47,3.0,0.0,5187.0,0.0,123.0,5187.0,47.0,110.361702,56.0,129.0,0.434109
2,American Airlines Inc.,11095,3708,769.0,58.0,294014.0,17561.0,15630.0,311575.0,3766.0,82.733670,6073.0,17168.0,0.353740
3,Delta Air Lines Inc.,3907,978,247.0,5.0,95326.0,741.0,5137.0,96067.0,983.0,97.728383,1724.0,5631.0,0.306162
4,Frontier Airlines Inc.,334,137,14.0,1.0,12098.0,114.0,486.0,12212.0,138.0,88.492754,180.0,514.0,0.350195


In [367]:
# Get Percents

summary['ON_TIME_PERC'] = summary['ON_TIME'] / summary['ALL_FLIGHTS']
summary['DELAYED_PERC'] = summary['DELAYED'] / summary['ALL_FLIGHTS']
summary['CANCELLED_PERC'] = summary['CANCELLED'] / summary['ALL_FLIGHTS']
summary['DIVERTED_PERC'] = summary['DIVERTED'] / summary['ALL_FLIGHTS']
summary['UNRELIABLE_ADJ_PERC'] = summary['UNRELIABLE_ADJ'] / summary['ALL_FLIGHTS_ADJ']


summary.head()

,CARRIER_DETAIL,ON_TIME,DELAYED,CANCELLED,DIVERTED,ARRIVAL_DELAY_TOTAL,DIVERSION_DELAY_TOTAL,ALL_FLIGHTS,TOTAL_DELAY,DEL_DIV,DEL_DIV_AVG,UNRELIABLE_ADJ,ALL_FLIGHTS_ADJ,ALL_FLIGHTS_ADJ_PERC,ON_TIME_PERC,DELAYED_PERC,CANCELLED_PERC,DIVERTED_PERC,UNRELIABLE_ADJ_PERC
0,Alaska Airlines Inc.,506,197,7.0,8.0,14521.0,1310.0,718.0,15831.0,205.0,77.224390,226.0,732.0,0.308743,0.704735,0.274373,0.009749,0.011142,0.308743
1,Allegiant Air,73,47,3.0,0.0,5187.0,0.0,123.0,5187.0,47.0,110.361702,56.0,129.0,0.434109,0.593496,0.382114,0.024390,0.000000,0.434109
2,American Airlines Inc.,11095,3708,769.0,58.0,294014.0,17561.0,15630.0,311575.0,3766.0,82.733670,6073.0,17168.0,0.353740,0.709853,0.237236,0.049200,0.003711,0.353740
3,Delta Air Lines Inc.,3907,978,247.0,5.0,95326.0,741.0,5137.0,96067.0,983.0,97.728383,1724.0,5631.0,0.306162,0.760561,0.190383,0.048083,0.000973,0.306162
4,Frontier Airlines Inc.,334,137,14.0,1.0,12098.0,114.0,486.0,12212.0,138.0,88.492754,180.0,514.0,0.350195,0.687243,0.281893,0.028807,0.002058,0.350195


In [368]:
# Carrier Ranked By On-time Percent

summary[['CARRIER_DETAIL', 'ON_TIME_PERC']].sort_values(by = 'ON_TIME_PERC', ascending = False)

,CARRIER_DETAIL,ON_TIME_PERC
3,Delta Air Lines Inc.,0.760561
8,United Air Lines Inc.,0.726244
2,American Airlines Inc.,0.709853
0,Alaska Airlines Inc.,0.704735
7,Spirit Air Lines,0.688761
4,Frontier Airlines Inc.,0.687243
5,JetBlue Airways,0.653937
6,Southwest Airlines Co.,0.633823
1,Allegiant Air,0.593496


In [369]:
# Carrier Ranked By Delayed Percent

summary[['CARRIER_DETAIL', 'DELAYED_PERC']].sort_values(by = 'DELAYED_PERC', ascending = True)

,CARRIER_DETAIL,DELAYED_PERC
3,Delta Air Lines Inc.,0.190383
8,United Air Lines Inc.,0.231863
2,American Airlines Inc.,0.237236
0,Alaska Airlines Inc.,0.274373
7,Spirit Air Lines,0.276657
4,Frontier Airlines Inc.,0.281893
5,JetBlue Airways,0.308461
6,Southwest Airlines Co.,0.341865
1,Allegiant Air,0.382114


In [370]:
# Carrier Ranked By Cancelled Percent

summary[['CARRIER_DETAIL', 'CANCELLED_PERC']].sort_values(by = 'CANCELLED_PERC', ascending = True)

,CARRIER_DETAIL,CANCELLED_PERC
0,Alaska Airlines Inc.,0.009749
6,Southwest Airlines Co.,0.021246
1,Allegiant Air,0.024390
7,Spirit Air Lines,0.025937
4,Frontier Airlines Inc.,0.028807
5,JetBlue Airways,0.031140
8,United Air Lines Inc.,0.038911
3,Delta Air Lines Inc.,0.048083
2,American Airlines Inc.,0.049200


In [371]:
# Carrier Ranked By Route Diverted Percent (Only Includes Flights That Make It To Expected Final Destination)

summary[['CARRIER_DETAIL', 'DIVERTED_PERC']].sort_values(by = 'DIVERTED_PERC', ascending = True)

,CARRIER_DETAIL,DIVERTED_PERC
1,Allegiant Air,0.000000
3,Delta Air Lines Inc.,0.000973
4,Frontier Airlines Inc.,0.002058
8,United Air Lines Inc.,0.002981
6,Southwest Airlines Co.,0.003066
2,American Airlines Inc.,0.003711
5,JetBlue Airways,0.006463
7,Spirit Air Lines,0.008646
0,Alaska Airlines Inc.,0.011142


In [372]:
# Carrier Ranked By Avg Delay / Diversion in Minutes

summary[['CARRIER_DETAIL', 'DEL_DIV_AVG']].sort_values(by = 'DEL_DIV_AVG', ascending = True)

,CARRIER_DETAIL,DEL_DIV_AVG
6,Southwest Airlines Co.,56.125672
0,Alaska Airlines Inc.,77.224390
2,American Airlines Inc.,82.733670
4,Frontier Airlines Inc.,88.492754
8,United Air Lines Inc.,88.765299
5,JetBlue Airways,90.652985
7,Spirit Air Lines,94.673401
3,Delta Air Lines Inc.,97.728383
1,Allegiant Air,110.361702


In [373]:
# Carrier Ranked By Unreliable Percent (Only Includes Flights That Make It To Expected Final Destination)

summary[['CARRIER_DETAIL', 'UNRELIABLE_ADJ_PERC']].sort_values(by = 'UNRELIABLE_ADJ_PERC', ascending = True)

,CARRIER_DETAIL,UNRELIABLE_ADJ_PERC
3,Delta Air Lines Inc.,0.306162
0,Alaska Airlines Inc.,0.308743
8,United Air Lines Inc.,0.326193
7,Spirit Air Lines,0.345205
4,Frontier Airlines Inc.,0.350195
2,American Airlines Inc.,0.353740
5,JetBlue Airways,0.384403
6,Southwest Airlines Co.,0.392012
1,Allegiant Air,0.434109
